In [5]:
def mult(x, y):
    assert y >= 0
    k = 0
    z = 0
    while k < y:
        assert z == k * x
        k += 1
        z += x
        assert z == k * x
    return z

In [ ]:
def intdiv(x ,y):
    assert x >= 0
    assert y > 0
    while r >= y:
        r -= y

In [6]:
mult(3,5)

15

In [36]:
def fac(n):
    if n > 1:
        return n * fac(n-1)
    else:
        return 1

In [37]:
%%time
fac(150)

CPU times: user 31 μs, sys: 22 μs, total: 53 μs
Wall time: 57.2 μs


57133839564458545904789328652610540031895535786011264182548375833179829124845398393126574488675311145377107878746854204162666250198684504466355949195922066574942592095735778929325357290444962472405416790722118445437122269675520000000000000000000000000000000000000

In [38]:
def fac(n):
    k = 0
    f = 1
    while k < n:
        k += 1
        f *= k
    return f

In [39]:
%%time
fac(150)

CPU times: user 20 μs, sys: 1 μs, total: 21 μs
Wall time: 23.8 μs


57133839564458545904789328652610540031895535786011264182548375833179829124845398393126574488675311145377107878746854204162666250198684504466355949195922066574942592095735778929325357290444962472405416790722118445437122269675520000000000000000000000000000000000000

In [41]:
import numpy as np

In [42]:
# create a numpy array with random number between 0 and 1:
arr = np.random.rand(100)

In [43]:
arr

array([0.95871809, 0.50431525, 0.312198  , 0.65707839, 0.61826706,
       0.13469539, 0.2880884 , 0.36055625, 0.3947069 , 0.03281793,
       0.22253951, 0.02600889, 0.48654752, 0.52286754, 0.64498851,
       0.10208027, 0.55811722, 0.4838407 , 0.63285058, 0.17730075,
       0.3983198 , 0.31474612, 0.27980401, 0.41558305, 0.58690247,
       0.50459756, 0.40284853, 0.69902083, 0.90138163, 0.51018693,
       0.90995718, 0.6287358 , 0.55954788, 0.29671631, 0.16083373,
       0.70988071, 0.90490407, 0.22362193, 0.67805226, 0.62600366,
       0.25565602, 0.48819371, 0.30034874, 0.20441551, 0.56935576,
       0.96950693, 0.88296689, 0.93950195, 0.49095827, 0.6296728 ,
       0.71923183, 0.81364468, 0.16426221, 0.77938285, 0.59014043,
       0.98039696, 0.32940962, 0.23042995, 0.45327409, 0.86708643,
       0.96855377, 0.65048685, 0.59767627, 0.36925174, 0.14859245,
       0.65153934, 0.7455679 , 0.31067714, 0.63672091, 0.60484334,
       0.77501894, 0.26430361, 0.48255351, 0.863149  , 0.37246

In [45]:
# compute variance of a given numpy array arr:
arr.var()

np.float64(0.06693050781917315)

In [ ]:
# define a function that computes the variance of a given numpy array
def compute_variance(arr):
    N = arr.size
    arr_mean = arr.sum() / N
    
    
    return squared_diffs.sum() / N

In [50]:
compute_variance(arr)

np.float64(0.06693050781917315)

In [ ]:
def runavg(data, W):
    """Compute a running average of a given dataset.

    Parameters
    ----------
    data: numpy.ndarray
        The input array for which the running average is to be computed.
    W: int
        The window size for computing the running average.

    Returns
    -------
    numpy.ndarray
        An array containing the running averages.
    """
    
    pass # todo